# TITLE: Text Classification using Naive Bayes


In [19]:
import numpy as np                                                  # to preform numerical operations
from sklearn import datasets                                        # to import required dataset
from sklearn import model_selection                                 # to split data into training and testing data
from nltk.corpus import stopwords                                   # for removing stopwords
import itertools                                                    # for slicing vocabulary dictionary into required length
import  pandas as pd                                                # for dataset manipulation
from sklearn.metrics import classification_report, confusion_matrix # for getting info of result precision and accuracy
from nltk.tokenize import word_tokenize                             # for tokenizing  
import math as ma
import nltk

In [20]:
stop_words=set(stopwords.words('english'))                                            # get the stopwords 
news=datasets.fetch_20newsgroups()                                                    # load data into news 
x_train,x_test,y_train,y_test=model_selection.train_test_split(news.data,news.target,test_size=0.25) # split data into test and train

In [9]:
for i in range(len(x_train)):                                       #converting train data in lower form
    x_train[i] = x_train[i].lower()
for i in range(len(x_test)):                                        #converting test data in lower form.
    x_test[i]=x_test[i].lower()

In [10]:
print(stop_words)                                                   # Just checking what are the stopwords


{'what', 'while', 'd', 'myself', 'just', 'ours', 'mustn', 'so', "that'll", 'above', 'whom', 'this', 'you', "couldn't", 'in', 'about', 'has', 'ma', 'no', 'which', 'all', "weren't", 'those', 'should', 'needn', 'my', 'very', 'yours', "she's", 'been', 'haven', 'there', 'into', 'be', 'by', 'some', 'am', 've', 'at', 'through', 'y', "needn't", "mustn't", 'own', 'don', 'as', 'with', 'aren', "isn't", 'wouldn', 'them', 'were', 'hers', "wasn't", 'was', 'we', 'do', 'yourselves', 'and', "should've", 'that', 'did', 'because', 'more', "won't", 'ourselves', 'had', 'for', 'again', 'before', 'who', 'after', "haven't", 'him', 'mightn', 'during', 'doesn', 'from', 'down', 'will', 'of', 'her', "you'll", 's', 'up', 'below', 'an', 'same', "didn't", 'their', 'wasn', 'other', 'then', 'itself', 'o', 'hadn', 'but', 'me', 'under', 'herself', 'being', 'our', 'it', 'where', 'most', 'his', 'weren', 'on', 'can', 'off', 'theirs', "hasn't", 'over', "wouldn't", 'further', "it's", 'between', 'nor', "you're", "you've", 'll

In [11]:
import nltk
nltk.download('punkt')                                             # needed to be done first time
d={}                                                                # d is going to be our vocabulary dictionary
for i in range(len(x_train)):                                       # this function filters stopwords and creates vocabulary                                                # taken one row at a time of training data
    data = x_train[i]
    words=word_tokenize(data)                                       # tokenize the data
    filtered_sentence=[ele for ele in words if ele not in stop_words] # get filtered sentence
    for ele in filtered_sentence:                                   # create vocabulary from filtered sentence    
        d[ele]=d.get(ele,0)+1                                         
new_dict={}                                                         # this will be our final vocabulary dictionary
for key,value in sorted(d.items(), key=lambda kv: kv[1],reverse=True): # put the key,value pair from d in decreasing order of value
    new_dict[key]=value
new_dict=dict(itertools.islice(new_dict.items(),6000))             # Take the top 6000 elements into final vocabulary
features=[]
for i in new_dict:
    features.append(i)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anush\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [12]:
print(new_dict)                                                     # just checking final created vocabulary dictionary
#print(features.shape)

{',': 122505, '.': 107035, '>': 103234, '--': 90364, ':': 85723, '(': 52999, ')': 51312, '@': 43486, '?': 25810, "''": 22999, '``': 20329, '<': 18263, '!': 17731, '|': 14025, "'ax": 13435, ';': 12723, "'s": 12612, '%': 12169, '#': 11924, '$': 11838, "n't": 11288, '-': 11073, ']': 10565, '`': 10327, '&': 9658, '[': 9312, 'subject': 9220, 'lines': 8855, '...': 8748, 'organization': 8412, "'": 7603, 'would': 7232, 'one': 6748, 'writes': 5859, 'article': 5063, '1': 4798, 'people': 4570, 'like': 4347, 'university': 4169, '*': 3952, 'know': 3905, 'get': 3783, 'nntp-posting-host': 3611, '2': 3562, 'x': 3535, 'think': 3418, 'also': 3229, 'use': 3119, 'new': 3001, 'time': 2997, 'could': 2938, '0': 2802, 'good': 2795, "'m": 2733, 'even': 2523, 'may': 2509, 'well': 2498, 'q': 2384, 'see': 2323, '3': 2319, 'us': 2270, 'two': 2222, 'many': 2212, 'first': 2207, 'make': 2206, '7': 2202, 'way': 2200, 'much': 2184, 'god': 2137, '+': 2135, '4': 2122, 'system': 2076, 'right': 2070, 'distribution': 2059, 

In [13]:
def updatedRow(words,features,i):                                   
    lst=np.array([0]*6000)                                         # Initialise the lst to be np.zeros with dimensions 1 X 6000                                                           # i will help us iterate our lst 
    for ele in words:
        if ele in features:
            lst[features.index(ele)]=words.count(ele)
    return lst

def updateXTrain(x_train,features):
    x_train_updated=pd.DataFrame(np.zeros((len(x_train),6000)))     # Intialise the np.zeros 2-D array to x_train_updated
    x_train_updated.columns=features                                # Add columns to the x_train_updated
    for i in range(len(x_train)):                                   # Now operate on each row
        curr_x=x_train[i]                                           # curr_x contains the row i content
        words=word_tokenize(curr_x)                                 # extract all words of curr_x into words
        x_train_updated.iloc[i,:]=updatedRow(words,features,i)      # replace row i with the updateRow() function's list
    return x_train_updated                                          # return x_train_updated

x_train_updated=updateXTrain(x_train,features)                      # get updated x_train_updated
x_train_updated.columns                                             # just checking the columns

Index([',', '.', '>', '--', ':', '(', ')', '@', '?', '''',
       ...
       'awards', 'beaten', 'headquarters', 'acquired', 'sophisticated',
       'implications', '919', 'utzoo', 'businesses', 'upgraded'],
      dtype='object', length=6000)

In [14]:
def fit(X_train, Y_train):                                          # This function is used to fit training data into our model
    result = {}                                                     # This dictionary is going to be useful in later calculations
    class_values = set(Y_train) 
    for current_class in class_values:                              # We create keys for all the possible classes
        result[current_class] = {}                                  # We create a dictionary as value for each key itself
        result["total"] = len(Y_train)                              # It gives total element present in our dictionary
        current_class_rows = (Y_train == current_class)             # Obtain rows for the current class
        X_train_current = X_train[current_class_rows]               # Filter the x_train for current class
        Y_train_current = Y_train[current_class_rows]               # Filter the y_train for current class
        num_features = X_train.shape[1]                             
        result[current_class]["count"] = len(Y_train_current)       # It gives total number of features of our data
        a=0                                                         # To get total number of a particular feature 
        for j in range(num_features):                               # traverse each feature
            result[current_class][j]=X_train_current[:,j].sum()     # Get total number of current feature
            a+=result[current_class][j]                             # Increment a by total number of current feature
        result[current_class]['total']=a                            # Assign a, will be used in later calculations
    return result                                                   # Return result


x_train_updated=np.array(x_train_updated)                           # Change pandas dataframe to numpy array
d=fit(x_train_updated,y_train)                                      # Fit ur trainig data

In [15]:
def probability(dictionary, x, current_class): 
    output = ma.log(dictionary[current_class]["total"]) - ma.log(dictionary["total"])
    num_features = len(dictionary[current_class].keys()) - 2
    for j in range( num_features ):
        if x[j]==0:
            continue
        count_current_class_with_value_j = dictionary[current_class][j]+1
        count_current_class = dictionary[current_class]['total']+num_features
        current_j_probablity = ma.log(count_current_class_with_value_j) - ma.log(count_current_class)
        output = output + current_j_probablity
    return output

def doSinglePrediction(x,dictionary):                                  # Function to predict class for a single row
    classes = dictionary.keys()                                        # Get all possible classes
    best_p = -100                                                      # Initialise best probablity & class to some -ve number
    best_class = -100 
    first_run = True                                                   # Running for first time = True
    for current_class in classes:                                      # Iterate over each possible class
        if (current_class == "total"):                                 # Ignore 'total' key
            continue
        p_current_class = probability(dictionary, x, current_class)    # Get probablity for x belonging to current class
        if (first_run or p_current_class > best_p):                    # If this is greatest probablity till now, change the
            best_p = p_current_class                                    # value of greatest probability & predicted class
            best_class = current_class
        first_run = False                                              # First run complete
    return best_class                                                  # Return the predicted class out of all classes

def y_predict(x_test,d,features):                                       # Function to predict the output values for test data
    y_pred=np.zeros(len(x_test))                                        # We will use this numpy array to store predictions
    x_test_updated=updateXTrain(x_test,features)                        # Update x_test so that our model can operate on it
    x_test_updated=np.array(x_test_updated)                             # Change pandas dataframe to numpy array
    for i in range(len(x_test)):                                                                                                        # Extract one-one row from data
        curr_x=x_test_updated[i,:]                                      # Extract one-one row 
        y_pred[i]=doSinglePrediction(curr_x,d)                          # Predict class for current row, store the prediction
    return y_pred                                                       # Return predictions

In [16]:
y_pred=y_predict(x_test,d,features)                                     # Call function to predict classes for x_test
print(classification_report(y_test,y_pred))                             # Print classification report
print(confusion_matrix(y_test,y_pred))                                  # Print confusion matrix

              precision    recall  f1-score   support

           0       0.91      0.89      0.90       132
           1       0.55      0.81      0.65       150
           2       0.80      0.05      0.10       147
           3       0.54      0.67      0.60       144
           4       0.57      0.84      0.68       124
           5       0.85      0.65      0.74       136
           6       0.68      0.82      0.75       142
           7       0.82      0.88      0.85       160
           8       0.88      0.89      0.89       160
           9       0.93      0.94      0.94       161
          10       0.99      0.90      0.94       148
          11       0.92      0.92      0.92       149
          12       0.75      0.76      0.76       151
          13       0.95      0.84      0.89       140
          14       0.87      0.91      0.89       135
          15       0.89      0.92      0.90       155
          16       0.93      0.82      0.87       147
          17       0.97    

In [17]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.8038176033934252


In [18]:
print('\n\n\t\tCOMPARING WITH LIBRARY IMPLEMENTATION')      # Compare built model with the sklearn's inbuilt model

from sklearn.naive_bayes import MultinomialNB               # Import MultinomialDB from sklearn
clf = MultinomialNB()                                       # Create object of MultinomialDB
clf.fit(x_train_updated, y_train)                           # Fit the training data
x_test_updated=updateXTrain(x_test,features)                # Update x_test so that model can operate on it
x_test_updated=np.array(x_test_updated)                     # Convert to np array
y_pred = clf.predict(x_test_updated)                        # Do predictions
print(classification_report(y_test,y_pred))                 # Print classification report
print(confusion_matrix(y_test,y_pred))                      # Print confusion matrix
print(accuracy_score(y_test,y_pred))




		COMPARING WITH LIBRARY IMPLEMENTATION
              precision    recall  f1-score   support

           0       0.84      0.89      0.86       132
           1       0.53      0.80      0.64       150
           2       0.58      0.07      0.13       147
           3       0.55      0.70      0.62       144
           4       0.64      0.77      0.70       124
           5       0.67      0.68      0.68       136
           6       0.67      0.82      0.74       142
           7       0.83      0.81      0.82       160
           8       0.78      0.93      0.85       160
           9       0.89      0.94      0.92       161
          10       0.96      0.84      0.90       148
          11       0.97      0.90      0.93       149
          12       0.74      0.71      0.72       151
          13       0.94      0.86      0.90       140
          14       0.94      0.87      0.91       135
          15       0.91      0.88      0.90       155
          16       0.70      0.83      